In [112]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
import numpy as np
import nltk
import pandas as pd
from ast import literal_eval
from collections import Counter
from scipy.optimize import fmin_l_bfgs_b
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import multiprocessing
import random
from tqdm import tqdm
import optimizeTopicVectors as ot

In [113]:
# hoseong's class
from STMD import *
from ASUM import *
from ASUM_Embedding import *
from preprocess import *

In [42]:
# load data
# work_path = "/media/hs-ubuntu/data/dataset/MasterThesis/STMD_data/"
work_path = "E:/dataset/MasterThesis/STMD_data/"

data = pd.read_csv(work_path + "preprocess_complete_Electronics.csv")

In [12]:
brand = ['Apple', 'Samsung','Canon']
brand_df = data[data.brand.isin(brand)]
brand_df.reset_index(drop=True, inplace=True)
#긍정, 부정 반반씩
pos_reviews = brand_df[brand_df.overall >= 4]
neg_reviews = brand_df[brand_df.overall <= 2]
pos_sample = pos_reviews.sample(3500, random_state=23)
neg_sample = neg_reviews.sample(3500, random_state=42)
df = pd.concat([pos_sample, neg_sample], axis=0)
df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
df.reset_index(drop=True, inplace=True)

In [14]:
# 긍/부정 ratio
print(pos_sample.shape)
print(neg_sample.shape)

(3500, 12)
(3500, 12)


In [39]:
# prepare
sentence_list, sentiment_label, sentence_senti_label, \
pos_neg_sentence_indices, pos_neg_sentiment_label, numSentence = prepare(df)

documents, sentence_list_again, bigram, documents_label\
= bigram_and_sentence(sentence_senti_label, sentence_list, numSentence, max_vocab=5000, threshold = 5, min_count = 20)

In [54]:
documents[7000]

TaggedDocument(words=['ye', 'know', 'camera', 'crop_sensor', 'pleas', 'prime', 'lower', 'end', 'take', 'better', 'qualiti', 'photo', 'thing'], tags=['negative'])

In [43]:
# word / sentiment embedding

window = [2]
size = [100]
passes = 20
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, 
                        dm_mean=1, 
                        negative =10,
                        min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
            if (epoch + 1) % 5 ==0:
                model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [06:14<00:00, 19.41s/it]


In [45]:
model = Doc2Vec.load(model_path + 'model_2_100_10')

In [48]:
wordVectors = np.zeros((len(model.index2word), model.vector_size))
for index, word in enumerate(model.index2word):
    wordVectors[index,:] = model[word]

sentimentVector = np.zeros((2, model.vector_size))
sentimentVector[0,:] = model.docvecs['positive']
sentimentVector[1,:] = model.docvecs['negative']

In [94]:
%%time

asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=2, save_iter = 3, maxIters= 3)

Starting iteration 2 of 3
0.792
Starting save model
Wall time: 1min 32s


In [53]:
stmd = STMD(wordVectors, sentimentVector, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
stmd._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
stmd.run(sentence_list_again, print_iter=2, maxIters= 10)

Starting iteration 2 of 10
0.585428571429
Starting iteration 4 of 10
0.615428571429
Starting iteration 6 of 10
0.614571428571
Starting iteration 8 of 10
0.605714285714
Starting iteration 10 of 10
0.602428571429


In [93]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_"
asum = ASUM(wordVectors, sentimentVector, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 3, maxIters= 3)

Starting iteration 2 of 3
0.789
Starting save model
Wall time: 1min 13s


In [89]:
%%time


Starting iteration 2 of 3
0.792285714286
Starting save model
Wall time: 1min 29s


In [117]:
#parmeter search
model_path = "E:/dataset/MasterThesis/gensim_models/"
asum_embedding_path = "E:/dataset/MasterThesis/Models/ASUM_embedding"

window = [2,3,5]
size = [100,200,300]
num_topics = [2,5,10,20,50,100]
passes = 20
for w in window:
    for s in size:
        model = Doc2Vec(dm=1, dm_mean=1, negative =10,min_count=0, sample=1e-5,
                        window=w, size=s, 
                        workers=multiprocessing.cpu_count(), 
                        alpha=0.025, min_alpha=0.025)
        model.build_vocab(documents)

        for epoch in tqdm(range(passes)):
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= 0.002  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
            if (epoch + 1) % 5 ==0:
                model.save(model_path + 'model_' + str(w) + '_' + str(s) + '_' + str(epoch+1))
                
                # asum embedding model
                wordVectors = np.zeros((len(model.index2word), model.vector_size))
                for index, word in enumerate(model.index2word):
                    wordVectors[index,:] = model[word]

                sentimentVector = np.zeros((2, model.vector_size))
                sentimentVector[0,:] = model.docvecs['positive']
                sentimentVector[1,:] = model.docvecs['negative']
                for k in num_topics:
                    print("num_of_topic : %s, size : %s, window : %s, epoch : %s, completed!!!"%(k,s,w,epoch+1))
                    asum_embedding = ASUM_Embedding(wordVectors, sentimentVector, numTopics=k, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
                    asum_embedding._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
                    asum_embedding.run(sentence_list_again, save_path=asum_embedding_path, print_iter=10, save_iter = 20, maxIters= 100)


 20%|████████████████▌                                                                  | 4/20 [01:20<05:19, 19.94s/it]

size : 100, window : 2, epoch : 5, completed!!!
--- topic modeling start, num of topic : 2----


KeyboardInterrupt: 